1. Project Description
Use Case: "Golden Gate Bridge SHM Assistant"
Goal: Structural Health Monitoring (SHM) systems on bridges generate massive amounts of high-velocity data from accelerometers, strain gauges, and weather stations. Civil engineers cannot manually review every data point.
In this demo, we simulate a Bridge IoT System that aggregates hourly sensor averages. A Generative AI model is used to act as a "Virtual Safety Inspector," analyzing the raw JSON data to produce a daily engineering report that flags anomalies (like excessive vibration during high winds).

Workflow:

Simulate Data: Create a JSON object representing 24 hours of sensor readings from the bridge.

Prompt Engineering: Configure the AI to act as a Senior Structural Engineer.

API Call: Send the sensor data to the model.

Output: A professional safety assessment with a "Risk Level" determination.

2. Setup and API Connection
The OpenAI API (e.g. GPT-4o-mini) is used via the Python SDK. This model is efficient for processing structured text data (JSON) and identifying patterns.

In [ ]:
! pip install -U openai

In [ ]:
! pip install python-dotenv

In [27]:
# Install the OpenAI library: !pip install -q -U openai

import json
import os
from dotenv import load_dotenv
from openai import OpenAI

# --- CONFIGURATION ---
# Load API key from .env file (copy .env.example to .env and add your key)
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError(
        "OPENAI_API_KEY not found. Copy .env.example to .env and set your key from https://platform.openai.com/api-keys"
    )

client = OpenAI(api_key=api_key)
# Model used for chat completions (e.g. gpt-4o-mini for speed/cost, gpt-4o for higher quality)
model_name = "gpt-4o-mini"

print("✅ Bridge Monitor AI configured and ready.")

✅ Bridge Monitor AI configured and ready.


3. Data Simulation (The "IoT" Part)
This cell simulates the data collected by the bridge's IoT gateway. We will introduce a scenario where high winds caused elevated vibrations, but the structural integrity (strain) remained safe.

In [28]:
# Simulate 24 hours of bridge sensor data (hourly averages)
# Scenario: High winds caused elevated vibrations; strain remained within safe limits.
sensor_data = {
    "bridge": "Golden Gate Bridge",
    "date": "2025-02-11",
    "readings": [
        {"hour": h, 
         "wind_speed_kmh": 15 + (10 if 8 <= h <= 18 else 0), 
         "vibration_mmg": 0.2 + (0.5 if 10 <= h <= 16 else 0), 
         "strain_microstrain": 120 + (20 if 12 <= h <= 14 else 0), 
         "temp_c": 14 + (h % 6)
        }
        for h in range(24)
    ]
}
json_input = json.dumps(sensor_data, indent=2)
print("Sample sensor data (first 6 hours):")
print(json.dumps({**sensor_data, "readings": sensor_data["readings"][:6]}, indent=2))

Sample sensor data (first 6 hours):
{
  "bridge": "Golden Gate Bridge",
  "date": "2025-02-11",
  "readings": [
    {
      "hour": 0,
      "wind_speed_kmh": 15,
      "vibration_mmg": 0.2,
      "strain_microstrain": 120,
      "temp_c": 14
    },
    {
      "hour": 1,
      "wind_speed_kmh": 15,
      "vibration_mmg": 0.2,
      "strain_microstrain": 120,
      "temp_c": 15
    },
    {
      "hour": 2,
      "wind_speed_kmh": 15,
      "vibration_mmg": 0.2,
      "strain_microstrain": 120,
      "temp_c": 16
    },
    {
      "hour": 3,
      "wind_speed_kmh": 15,
      "vibration_mmg": 0.2,
      "strain_microstrain": 120,
      "temp_c": 17
    },
    {
      "hour": 4,
      "wind_speed_kmh": 15,
      "vibration_mmg": 0.2,
      "strain_microstrain": 120,
      "temp_c": 18
    },
    {
      "hour": 5,
      "wind_speed_kmh": 15,
      "vibration_mmg": 0.2,
      "strain_microstrain": 120,
      "temp_c": 19
    }
  ]
}


4. Interaction with the Model
Here we define the Prompt. We instruct the AI to analyze the correlation between the environmental factors (wind) and the structural response (vibration/displacement).

In [29]:
# Define the prompt acting as a Senior Engineer
prompt = f"""
You are an expert Structural Health Monitoring (SHM) Engineer AI.
Your task is to review the daily sensor logs for the Golden Gate Bridge and produce a safety report.

Instructions:
1. **Analyze:** Correlate the environmental data (wind/weather) with the structural response (vibration/displacement).
2. **Assess Risk:** Assign an overall Risk Level (Green, Yellow, or Red).
3. **Summarize:** Provide a brief summary of the bridge's health for the maintenance team.
4. **Action Items:** Recommend any necessary inspections.

Here is the raw sensor data:
{json_input}
"""

# Generate the content
try:
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
    )
    print("\n--- 🏗️ STRUCTURAL HEALTH REPORT ---\n")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Error communicating with API: {e}")


--- 🏗️ STRUCTURAL HEALTH REPORT ---

### Safety Report for Golden Gate Bridge
**Date:** 2025-02-11

---

**1. Analyze:**
The daily readings indicate a consistent wind speed of 15 km/h for the first part of the day (00:00 - 07:00), which later increased to 25 km/h from 08:00 onwards. During the entire monitoring period, the structural response showed systematic vibrations and strain readings. 

- **Wind Speed Correlation:**
  - For hours 0-7, wind speed was stable at 15 km/h, resulting in low vibration levels (0.2 mmg) and consistent strain (120 microstrain).
  - At 08:00, the wind speed increased to 25 km/h. This change did not immediately affect the vibration readings; however, at hours 10-16, vibrations increased to 0.7 mmg. Notably, the strain increased to 140 microstrain then returned to 120 microstrain.
  - The combination of wind speed and vibration levels remained within acceptable limits.

**2. Assess Risk:**
- **Overall Risk Level:** **Yellow**
  - The data suggest that the b